In [1]:
import requests
import re
import json
import time
from requests.exceptions import ReadTimeout,ConnectionError

In [2]:
def saveTotxt(disNum, index, date_times, users, contents, title):
    with open('data/' + title + '.txt', 'a+', encoding='utf-8') as f:
        for i in range(date_times.__len__()):
            f.write(str(disNum) + ',')
            f.write(date_times[i] + ',')
            f.write(users[i] + ',')
            content = contents[i].replace(',','，')
            content = content.replace('\n','')
            f.write(content + '\n')
            disNum += 1
            index += 1
        f.close()
    return disNum, index

In [3]:
def is_json(myjson):
    try:
        json.loads(myjson)
    except ValueError:
        return False
    return True

In [4]:
def getDiscussionFromOnePage(url, headers, proxies, disNum, index, title):
    readTimeout = False
    readTimeout_count = 0
    while True:
        try:
            response = requests.get(url, headers=headers, timeout = 20)
            data_content = response.text[response.text.find('(')+1:-1]
            break
        except ReadTimeout:
            print('Get a timeout')
            readTimeout_count += 1
        except ConnectionError:
            print('Get a timeout')
            readTimeout_count += 1
        if readTimeout_count == 5:
            readTimeout = True
            break
            
    if readTimeout == True:
        print('Always timeout. Shiped current page')
    else:
        if response.status_code == 200 and is_json(data_content):
            jsonData = json.loads(data_content)
            date_times = []
            users = []
            contents = []
            for i in range(int(jsonData['data']['oriretnum'])):
                user_id = jsonData['data']['oriCommList'][i]['userid']
                user = jsonData['data']['userList'][user_id]['nick']
                timeS = jsonData['data']['oriCommList'][i]['time']
                timeArray = time.localtime(int(timeS))
                date_time = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
                content = jsonData['data']['oriCommList'][i]['content']
                users.append(user)
                contents.append(content)
                date_times.append(date_time)
            return saveTotxt(disNum, index, date_times, users, contents, title)
        else:
            print(str(response.status_code))
            print('Unknow ERROR')
            exit()

In [5]:
def getAUrlByCursor(old_url, pageCursor):
    url_split = old_url.split('&')
    new_url = ''
    for i in range(url_split.__len__()):
        if url_split[i].find('cursor=') == -1 and i < url_split.__len__()-1:
            new_url += url_split[i] + '&'
        elif url_split[i].find('cursor=') == -1 and i == url_split.__len__()-1:
            new_url += url_split[i]
        else:
            new_url += 'cursor=' + str(pageCursor) + '&'
    return new_url

In [6]:
def getNewsDiscussion(url, disNum, title):
    proxies = {}
    headers = {
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Connection': 'close',
        'Cookie': 'pgv_pvid=8784430618; tvfe_boss_uuid=f34f280f3582fa8d; ts_uid=3795007618; pgv_pvi=3382785024; pgv_info=ssid=s7178299300; pgv_si=s3334671360; g_tk=4dc2141a0e8e2815ed7ef64c3a31c134e1605985; _qpsvr_localtk=0.2919858913550557; uin=o0550868991; skey=@udQO6VXiL; pt2gguin=o0550868991; ptisp=cnc; RK=hWzoOu2BMx; ptcz=228d3db515bf88d1a33277ef19b679e01eebeb6c97f687bced2a73dbafd720d8; o_cookie=550868991; pac_uid=1_550868991; ad_play_index=113; ts_last=coral.qq.com/2660765453',
        'Host': 'coral.qq.com',
        'Referer': 'http://page.coral.qq.com/coralpage/comment/news.html',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.92 Safari/537.36'
    }
    pageNumFlag = False
    try:
        requests.adapters.DEFAULT_RETRIES = 5
        response = requests.get(url, headers=headers, timeout = 20)
        data_content = response.text[response.text.find('(')+1:-1]
        pageNumFlag = True
    except ReadTimeout:
        print('Failed. Get page num failed1')
    except ConnectionError:
        print('Failed. Get page num failed2')
    if pageNumFlag == True and response.status_code == 200 and is_json(data_content):
        jsonData = json.loads(data_content)
        pageCursor = jsonData['data']['first']
        count = jsonData['data']['oritotal']
        index = 0
        while True:
            url = getAUrlByCursor(url, pageCursor)
            disNum, index = getDiscussionFromOnePage(url, headers, proxies, disNum, index, title)
            print(str(index) + ' count finished (Total: ' + str(count) + ')')
            if index >= int(count):
                break
            pageCursor = jsonData['data']['last']
        print('Finished')
    else:
        print(str(response.status_code))
        print('Unknow ERROR')
        exit()
    return disNum

In [11]:
title = '范冰冰偷税漏税'
disNum = 1
url = 'http://coral.qq.com/article/2724941284/comment/v2?callback=_article2724941284commentv2&orinum=10&oriorder=o&pageflag=1&cursor=6408947285310173772&scorecursor=0&orirepnum=2&reporder=o&reppageflag=1&source=1&_=1540272921067'
disNum = getNewsDiscussion(url, disNum, title)
url = 'http://coral.qq.com/article/3168286124/comment/v2?callback=_article3168286124commentv2&orinum=10&oriorder=o&pageflag=1&cursor=6453072633163967004&scorecursor=0&orirepnum=2&reporder=o&reppageflag=1&source=1&_=1540272885832'
disNum = getNewsDiscussion(url, disNum, title)
url = 'http://coral.qq.com/article/3168724956/comment/v2?callback=_article3168724956commentv2&orinum=10&oriorder=o&pageflag=1&cursor=6453128226017607257&scorecursor=0&orirepnum=2&reporder=o&reppageflag=1&source=1&_=1540272853372'
disNum = getNewsDiscussion(url, disNum, title)
url = 'http://coral.qq.com/article/3169044446/comment/v2?callback=_article3169044446commentv2&orinum=10&oriorder=o&pageflag=1&cursor=6453152230104858230&scorecursor=0&orirepnum=2&reporder=o&reppageflag=1&source=1&_=1540272847274'
disNum = getNewsDiscussion(url, disNum, title)

10 count finished (Total: 7831)
20 count finished (Total: 7831)
30 count finished (Total: 7831)
40 count finished (Total: 7831)
50 count finished (Total: 7831)
60 count finished (Total: 7831)
70 count finished (Total: 7831)
80 count finished (Total: 7831)
90 count finished (Total: 7831)
100 count finished (Total: 7831)
110 count finished (Total: 7831)
120 count finished (Total: 7831)
130 count finished (Total: 7831)
140 count finished (Total: 7831)
150 count finished (Total: 7831)
160 count finished (Total: 7831)
170 count finished (Total: 7831)
180 count finished (Total: 7831)
190 count finished (Total: 7831)
200 count finished (Total: 7831)
210 count finished (Total: 7831)
220 count finished (Total: 7831)
230 count finished (Total: 7831)
240 count finished (Total: 7831)
250 count finished (Total: 7831)
260 count finished (Total: 7831)
270 count finished (Total: 7831)
280 count finished (Total: 7831)
290 count finished (Total: 7831)
300 count finished (Total: 7831)
310 count finished 

2470 count finished (Total: 7831)
2480 count finished (Total: 7831)
2490 count finished (Total: 7831)
2500 count finished (Total: 7831)
2510 count finished (Total: 7831)
2520 count finished (Total: 7831)
2530 count finished (Total: 7831)
2540 count finished (Total: 7831)
2550 count finished (Total: 7831)
2560 count finished (Total: 7831)
2570 count finished (Total: 7831)
2580 count finished (Total: 7831)
2590 count finished (Total: 7831)
2600 count finished (Total: 7831)
2610 count finished (Total: 7831)
2620 count finished (Total: 7831)
2630 count finished (Total: 7831)
2640 count finished (Total: 7831)
2650 count finished (Total: 7831)
2660 count finished (Total: 7831)
2670 count finished (Total: 7831)
2680 count finished (Total: 7831)
2690 count finished (Total: 7831)
2700 count finished (Total: 7831)
2710 count finished (Total: 7831)
2720 count finished (Total: 7831)
2730 count finished (Total: 7831)
2740 count finished (Total: 7831)
2750 count finished (Total: 7831)
2760 count fin

4890 count finished (Total: 7831)
4900 count finished (Total: 7831)
4910 count finished (Total: 7831)
4920 count finished (Total: 7831)
4930 count finished (Total: 7831)
4940 count finished (Total: 7831)
4950 count finished (Total: 7831)
4960 count finished (Total: 7831)
4970 count finished (Total: 7831)
4980 count finished (Total: 7831)
4990 count finished (Total: 7831)
5000 count finished (Total: 7831)
5010 count finished (Total: 7831)
5020 count finished (Total: 7831)
5030 count finished (Total: 7831)
5040 count finished (Total: 7831)
5050 count finished (Total: 7831)
5060 count finished (Total: 7831)
5070 count finished (Total: 7831)
5080 count finished (Total: 7831)
5090 count finished (Total: 7831)
5100 count finished (Total: 7831)
5110 count finished (Total: 7831)
5120 count finished (Total: 7831)
5130 count finished (Total: 7831)
5140 count finished (Total: 7831)
5150 count finished (Total: 7831)
5160 count finished (Total: 7831)
5170 count finished (Total: 7831)
5180 count fin

7310 count finished (Total: 7831)
7320 count finished (Total: 7831)
7330 count finished (Total: 7831)
7340 count finished (Total: 7831)
7350 count finished (Total: 7831)
7360 count finished (Total: 7831)
7370 count finished (Total: 7831)
7380 count finished (Total: 7831)
7390 count finished (Total: 7831)
7400 count finished (Total: 7831)
7410 count finished (Total: 7831)
7420 count finished (Total: 7831)
7430 count finished (Total: 7831)
7440 count finished (Total: 7831)
7450 count finished (Total: 7831)
7460 count finished (Total: 7831)
7470 count finished (Total: 7831)
7480 count finished (Total: 7831)
7490 count finished (Total: 7831)
7500 count finished (Total: 7831)
7510 count finished (Total: 7831)
7520 count finished (Total: 7831)
7530 count finished (Total: 7831)
7540 count finished (Total: 7831)
7550 count finished (Total: 7831)
7560 count finished (Total: 7831)
7570 count finished (Total: 7831)
7580 count finished (Total: 7831)
7590 count finished (Total: 7831)
7600 count fin

1860 count finished (Total: 25552)
1870 count finished (Total: 25552)
1880 count finished (Total: 25552)
1890 count finished (Total: 25552)
1900 count finished (Total: 25552)
1910 count finished (Total: 25552)
1920 count finished (Total: 25552)
1930 count finished (Total: 25552)
1940 count finished (Total: 25552)
1950 count finished (Total: 25552)
1960 count finished (Total: 25552)
1970 count finished (Total: 25552)
1980 count finished (Total: 25552)
1990 count finished (Total: 25552)
2000 count finished (Total: 25552)
2010 count finished (Total: 25552)
2020 count finished (Total: 25552)
2030 count finished (Total: 25552)
2040 count finished (Total: 25552)
2050 count finished (Total: 25552)
2060 count finished (Total: 25552)
2070 count finished (Total: 25552)
2080 count finished (Total: 25552)
2090 count finished (Total: 25552)
2100 count finished (Total: 25552)
2110 count finished (Total: 25552)
2120 count finished (Total: 25552)
2130 count finished (Total: 25552)
2140 count finished 

...

1840 count finished (Total: 8307)
1850 count finished (Total: 8307)
1860 count finished (Total: 8307)
1870 count finished (Total: 8307)
1880 count finished (Total: 8307)
1890 count finished (Total: 8307)
1900 count finished (Total: 8307)
1910 count finished (Total: 8307)
1920 count finished (Total: 8307)
1930 count finished (Total: 8307)
1940 count finished (Total: 8307)
1950 count finished (Total: 8307)
1960 count finished (Total: 8307)
1970 count finished (Total: 8307)
1980 count finished (Total: 8307)
1990 count finished (Total: 8307)
2000 count finished (Total: 8307)
2010 count finished (Total: 8307)
2020 count finished (Total: 8307)
2030 count finished (Total: 8307)
2040 count finished (Total: 8307)
2050 count finished (Total: 8307)
2060 count finished (Total: 8307)
2070 count finished (Total: 8307)
2080 count finished (Total: 8307)
2090 count finished (Total: 8307)
2100 count finished (Total: 8307)
2110 count finished (Total: 8307)
2120 count finished (Total: 8307)
2130 count fin

4260 count finished (Total: 8307)
4270 count finished (Total: 8307)
4280 count finished (Total: 8307)
4290 count finished (Total: 8307)
4300 count finished (Total: 8307)
4310 count finished (Total: 8307)
4320 count finished (Total: 8307)
4330 count finished (Total: 8307)
4340 count finished (Total: 8307)
4350 count finished (Total: 8307)
4360 count finished (Total: 8307)
4370 count finished (Total: 8307)
4380 count finished (Total: 8307)
4390 count finished (Total: 8307)
4400 count finished (Total: 8307)
4410 count finished (Total: 8307)
4420 count finished (Total: 8307)
4430 count finished (Total: 8307)
4440 count finished (Total: 8307)
4450 count finished (Total: 8307)
4460 count finished (Total: 8307)
4470 count finished (Total: 8307)
4480 count finished (Total: 8307)
4490 count finished (Total: 8307)
4500 count finished (Total: 8307)
4510 count finished (Total: 8307)
4520 count finished (Total: 8307)
4530 count finished (Total: 8307)
4540 count finished (Total: 8307)
4550 count fin

6670 count finished (Total: 8307)
6680 count finished (Total: 8307)
6690 count finished (Total: 8307)
6700 count finished (Total: 8307)
6710 count finished (Total: 8307)
6720 count finished (Total: 8307)
6730 count finished (Total: 8307)
6740 count finished (Total: 8307)
6750 count finished (Total: 8307)
6760 count finished (Total: 8307)
6770 count finished (Total: 8307)
6780 count finished (Total: 8307)
6790 count finished (Total: 8307)
6800 count finished (Total: 8307)
6810 count finished (Total: 8307)
6820 count finished (Total: 8307)
6830 count finished (Total: 8307)
6840 count finished (Total: 8307)
6850 count finished (Total: 8307)
6860 count finished (Total: 8307)
6870 count finished (Total: 8307)
6880 count finished (Total: 8307)
6890 count finished (Total: 8307)
6900 count finished (Total: 8307)
6910 count finished (Total: 8307)
6920 count finished (Total: 8307)
6930 count finished (Total: 8307)
6940 count finished (Total: 8307)
6950 count finished (Total: 8307)
6960 count fin

800 count finished (Total: 4238)
810 count finished (Total: 4238)
820 count finished (Total: 4238)
830 count finished (Total: 4238)
840 count finished (Total: 4238)
850 count finished (Total: 4238)
860 count finished (Total: 4238)
870 count finished (Total: 4238)
880 count finished (Total: 4238)
890 count finished (Total: 4238)
900 count finished (Total: 4238)
910 count finished (Total: 4238)
920 count finished (Total: 4238)
930 count finished (Total: 4238)
940 count finished (Total: 4238)
950 count finished (Total: 4238)
960 count finished (Total: 4238)
970 count finished (Total: 4238)
980 count finished (Total: 4238)
990 count finished (Total: 4238)
1000 count finished (Total: 4238)
1010 count finished (Total: 4238)
1020 count finished (Total: 4238)
1030 count finished (Total: 4238)
1040 count finished (Total: 4238)
1050 count finished (Total: 4238)
1060 count finished (Total: 4238)
1070 count finished (Total: 4238)
1080 count finished (Total: 4238)
1090 count finished (Total: 4238)


3230 count finished (Total: 4238)
3240 count finished (Total: 4238)
3250 count finished (Total: 4238)
3260 count finished (Total: 4238)
3270 count finished (Total: 4238)
3280 count finished (Total: 4238)
3290 count finished (Total: 4238)
3300 count finished (Total: 4238)
3310 count finished (Total: 4238)
3320 count finished (Total: 4238)
3330 count finished (Total: 4238)
3340 count finished (Total: 4238)
3350 count finished (Total: 4238)
3360 count finished (Total: 4238)
3370 count finished (Total: 4238)
3380 count finished (Total: 4238)
3390 count finished (Total: 4238)
3400 count finished (Total: 4238)
3410 count finished (Total: 4238)
3420 count finished (Total: 4238)
3430 count finished (Total: 4238)
3440 count finished (Total: 4238)
3450 count finished (Total: 4238)
3460 count finished (Total: 4238)
3470 count finished (Total: 4238)
3480 count finished (Total: 4238)
3490 count finished (Total: 4238)
3500 count finished (Total: 4238)
3510 count finished (Total: 4238)
3520 count fin